### Data Source from Yahoo_Finance

In [9]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime


In [15]:
a = yf.Ticker("TSLA")
a.history(start = '2017-01-01')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-01-03,42.972000,44.066002,42.192001,43.397999,29616500,0,0.0
2017-01-04,42.950001,45.599998,42.862000,45.397999,56067500,0,0.0
2017-01-05,45.284000,45.495998,44.389999,45.349998,29558500,0,0.0
2017-01-06,45.386002,46.062000,45.090000,45.801998,27639500,0,0.0
2017-01-09,45.793999,46.383999,45.599998,46.256001,19897500,0,0.0
...,...,...,...,...,...,...,...
2021-03-08,600.549988,620.130005,558.789978,563.000000,51497000,0,0.0
2021-03-09,608.179993,678.090027,595.210022,673.580017,67028000,0,0.0
2021-03-10,700.299988,717.849976,655.059998,668.059998,60486700,0,0.0


In [29]:
class data:
    def __init__(self, _ticker):
        self.ticker = yf.Ticker(_ticker)
        
    def get_Dailydata(self, _startdate):
        basic_data = self.ticker.history(start = _startdate)
        data = {
            'open': round(basic_data['Open'],2),
            'high': round(basic_data['High'],2),
            'low': round(basic_data['Low'],2),
            'close': round(basic_data['Close'],2),
            'volume': round(basic_data['Volume'],2)
        }
        df = pd.DataFrame(data)
        df = df.reset_index()
        df = df.rename(columns={'index': 'date'})
        df = df.iloc[1:].reset_index(drop = True)
        return df

In [30]:
# data source test
a = data("TSLA")
a.get_Dailydata('2015-01-01')

,Date,open,high,low,close,volume
0,2015-01-02,44.57,44.65,42.65,43.86,23822000
1,2015-01-05,42.91,43.30,41.43,42.02,26842500
2,2015-01-06,42.01,42.84,40.84,42.26,31309500
3,2015-01-07,42.67,42.96,41.96,42.19,14842000
4,2015-01-08,42.56,42.76,42.00,42.12,17212500
...,...,...,...,...,...,...
1554,2021-03-08,600.55,620.13,558.79,563.00,51497000
1555,2021-03-09,608.18,678.09,595.21,673.58,67028000
1556,2021-03-10,700.30,717.85,655.06,668.06,60486700
1557,2021-03-11,699.40,702.50,677.18,699.60,36106300


In [31]:
m = a.get_Dailydata('2015-01-01')
m.loc[0]['Date']

Timestamp('2015-01-02 00:00:00')

### back test frame
1. buy_sell robot 2. wallet 3. back test robot

In [32]:
class trade_Robot:

# Only to execute buy or sell action, return remain capital, current holding position
# currentPosition 现有的持仓股数
# trade_robot 从wallet获得最新的持仓情况（现有现金，现有股票持仓），
#             从backtest kernel获得当前股价，
#             从策略集合获得当前的操作
# 买卖操作返回的是（持有现金，持仓数目，股票现价，操作名称）


    def __init__(self, _ticker):
        self.ticker = _ticker
        
    def get_Currentwallet(self, _currentPrice, _currentCapital, _currentPosition):
        self.currentCapital = _currentCapital
        self.currentPosition = _currentPosition
        self.currentPrice = _currentPrice
        
    def buy(self, volume):
        if volume % 100 != 0:
            print('volume must be the multiple of 100')
            return 'Trade failed'
        if self.capital_Check('buy', volume):
            self.currentCapital = self.currentCapital - volume*self.currentPrice
            self.currentPosition = self.currentPosition + volume
            return self.currentCapital, self.currentPosition, self.currentPrice, self.ticker, 'Buy'
        else:
            print('Trade failed')
            return self.no_Action()
        
    def sell(self, volume):
        if volume % 100 != 0:
            print('volume must be the multiple of 100')
            return 'Trade failed'
        if self.capital_Check('sell', volume):
            self.currentCapital = self.currentCapital + volume*self.currentPrice
            self.currentPosition = self.currentPosition - volume
            return self.currentCapital, self.currentPosition, self.currentPrice, self.ticker, 'Sell'
        else:
            print('Trade failed')
            return self.no_Action()
        
    def no_Action(self):
        return self.currentCapital, self.currentPosition, self.currentPrice, self.ticker, 'No Action'
    
    def capital_Check(self, action, volume):
        if action == 'buy':
            if self.currentCapital >= volume*self.currentPrice:
                return True
            else:
                print('Insufficient current capital')
                return False
        if action == 'sell':
            if self.currentPosition >= volume:
                return True
            else:
                print('Insufficient holding position')
                return False   

In [33]:
t = trade_Robot("AAPL")
t.get_Currentwallet(10, 10000, 400)

In [34]:
a = t.sell(400)
a

(14000, 0, 10, 'AAPL', 'Sell')

In [59]:
class wallet:
    
    # store the data of current bought tickers, 
    # current capital, current hoding position, current share value, historical data
    # for now, the wallet only allow single ticker
    
    def __init__(self, _ticker, _initialCapital, _startdate):
        self.initialCapital = _initialCapital
        self.startdate = _startdate
        self.startIndex = 0
        self.ticker = _ticker
        self.shares_Data = pd.DataFrame([],
             columns = ['date', 
                        'ticker',
                        'remainCapital',
                        'currentPosition',
                        'currentShareprice',
                        'currentSharesValue',
                        'currentTotalCapital'
                       ])

    def initialization(self):
        self.shares_Data = pd.DataFrame({
            'date': [(self.startdate - datetime.timedelta(days=1))],
            'ticker': [self.ticker],
            'remainCapital': [self.initialCapital],
            'currentPosition': [0],
            'currentShareprice': [0],
            'currentSharesValue': [0],
            'currentTotalCapital': [self.initialCapital]
        })
        self.currentIndex = self.startIndex
        return
    
    def update_Wallet(self, currentDate, updatedCapital, updatedPosition, currentPrice):
        self.currentIndex += 1
        self.shares_Data.loc[self.currentIndex] = [
            currentDate,
            self.ticker,
            updatedCapital,
            updatedPosition,
            currentPrice,
            updatedPosition*currentPrice,
            updatedCapital + updatedPosition*currentPrice
        ]
        return
    
    def current_stat(self):
        # get current wallet stat
        data = self.shares_Data.loc[self.currentIndex][:]
        return data
    
    def historical_stat(self):
        data = self.shares_Data
        return data

In [60]:
import datetime
date = datetime.datetime.strptime('2021-03-07','%Y-%m-%d')
a = wallet('AAPL', 10000, date)

In [61]:
a.initialization()
a.shares_Data

,date,ticker,remainCapital,currentPosition,currentShareprice,currentSharesValue,currentTotalCapital
0,2021-03-06,AAPL,10000,0,0,0,10000


In [68]:
a.update_Wallet(datetime.datetime.strptime('2021-03-09','%Y-%m-%d'), 6000, 400, 25)

In [69]:
a.current_stat()

date                   2021-03-09 00:00:00
ticker                                AAPL
remainCapital                         6000
currentPosition                        400
currentShareprice                       25
currentSharesValue                   10000
currentTotalCapital                  16000
Name: 3, dtype: object

In [70]:
a.historical_stat()

,date,ticker,remainCapital,currentPosition,currentShareprice,currentSharesValue,currentTotalCapital
0,2021-03-06,AAPL,10000,0,0,0,10000
1,2021-03-07,AAPL,6000,400,20,8000,14000
2,2021-03-08,AAPL,6000,400,21,8400,14400
3,2021-03-09,AAPL,6000,400,25,10000,16000


In [206]:
d = a.current_stat()

In [207]:
d

date                   2021-03-06 00:00:00
ticker                                AAPL
remainCashflow                       10000
currentPosition                          0
currentShareprice                        0
currentSharesValue                       0
currentTotalCapital                  10000
Name: 0, dtype: object

In [ ]:
class backtest:
    def __init__(self, _ticker):